# Modulo di terze parti `requests`: effettuare HTTP requests

Questa è un'introduzione a `requests`, un'elegante e semplice libreria HTTP per Python.

L'Hypertext Transfer Protocol (HTTP) è probabilmente il protocollo per il trasferimento dati più diffuso su Internet. Consente la comunicazione tra il client (solitamente un browser web) e il server HTTP (server web) mediante l'invio di messaggi di testo: il client invia un messaggio di richiesta al server che, a sua volta, restituisce un messaggio di risposta.

### Info e installazione di `requests`

- [Sito e documentazione ufficiale](https://requests.readthedocs.io/projects/it/it/latest/index.html).
- [Pagina su PyPi](https://pypi.org/project/requests/) (Python Package Index).
- [Repo su GitHub](https://github.com/psf/requests).

Python `requests` consente di inviare tutti i tipi di richieste e di ottenere le relative risposte in modo semplice e intuitivo.

Questa è una libreria di terze parti e dunque non fa parte della Libreria standard. È necessario installarla tramite il comando `pip install requests` nella riga di comando.

```bash
# MAC/LINUX:
(my_venv) $ pip install requests

# WINDOWS:
(my_venv) C:\my_proj> pip install requests
```


Se sei su Windows, non hai creato un virtual environment (male!;) e devi usare il `py` launcher:

```powershell
C:\my_proj> py -m pip install requests
```

Per iniziare a usare requests nel vostro codice, importate la libreria:

```python
import requests
```

### Invio e recezione dei dati

Come detto, il protocollo HTTP prevede una serie di [messaggi di richiesta](https://it.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Messaggio_di_richiesta) che il client può inviare al server. In particolare i più noti sono

- GET
- POST
- PUT
- DELETE

La richiesta GET è usata per recuperare informazioni da un determinato server usando un URL. Per esempio, ogni volta che si inserisce un URL nella casella degli indirizzi del browser, questo lo traduce in un messaggio di richiesta GET e lo invia al server.

Il più delle volte navighiamo sul web recuperando alcuni dati, ma quando ci colleghiamo a un social network, aggiungiamo un articolo al carrello in un negozio online o cancelliamo una foto pubblicata in precedenza, inviamo informazioni al server per modificarne lo stato. A questo scopo utilizziamo metodi HTTP come POST, PUT e DELETE.

In questa sezione, vedremo come inviare questi tipi comuni di richieste con Python, limitandoci a GET e POST, che sono i metodi usati per reperire informazioni. PUT e DELETE vengono usati in casi specifici che non ci interessano ai fini di questo corso di base.

## Richiesta GET: `requests.get()`

Immaginiamo di dover ottenere la pagina principale del sito ufficiale della libreria `requests`. Possiamo farlo con l'aiuto di `requests.get(url)` che ci restituisce un oggetto di risposta (*response*) contenente tutte le informazioni riguardanti la risposta da parte del server.

Nell'esempio che segue otterremo un codice numerico che ci indicherà se la richiesta è andata a buon fine oppure no. Per esempio `200` indica che richiesta è stata ricevuta e non si sono verificati errori, mentre un codice `404` significa che la risorsa richiesta non è stata trovata. Questi numeri sono detti [codici di stato HTTP](https://it.wikipedia.org/wiki/Codici_di_stato_HTTP) e seguono una convenzione, uno [standard](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

È possibile accedere esplicitamente al codice di risposta tramite l'attributo `.status_code` dell'oggetto di risposta:

In [1]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

print(response)
print(response.status_code)


<Response [200]>
200


Il significato primario dei codici è determinato dalla prima cifra, secondo questo schema:

- `1xx`: *information* - La richiesta è stata ricevuta, proceda chi deve procedere, attenda chi deve attendere.
- `2xx`: *success* - La richiesta è stata ricevuta con successo, compresa ed accettata.
- `3xx`: *redirect* - Il client deve eseguire ulteriori azioni per completare la richiesta.
- `4xx`: *client error* - La richiesta contiene una sintassi errata o non può essere soddisfatta.
- `5xx`: *server error* - Il server non è riuscito a soddisfare una richiesta apparentemente valida.

Se si usa un oggetto response con il costrutto `if`, la valutazione sarà `True` se il codice o di tipo `1xx`, `2xx`, `3xx`, altrimenti sarà `False`:

- `1xx`, `2xx`, `3xx` &rarr; truthy
- `4xx`, `5xx` &rarr; falsy

In [3]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

if response:
    print('Success!')
else:
    print('An error has occurred.')


An error has occurred.


Per leggere il contenuto della risposta del server, si deve osservare la proprietà `.text`:

In [2]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

print(response.request.headers)

{'User-Agent': 'python-requests/2.28.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


Si noti che le richieste decodificano automaticamente il contenuto della risposta del server. È possibile conoscere la codifica utilizzata e modificarla, se necessario, utilizzando la proprietà `.encoding`:

In [7]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

response.encoding

'utf-8'

## Headers

Sia la richiesta sia la risposta oltre che a un "contenuto" vengono accompagnati anche da un cosiddetto _**header**_, ovvero una "intestazione".

In parole povere un [header HTTP](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields) è una sorta di bolla di accompagnamento del messaggio, che contiene dei dati utili ai fini della corretta comunicazione tra client e server sotto forma di coppie *campo* (chiave) e *valore*.

Più precisamente, *campi* dell'header HTTP sono un elenco di parole chiave e valori sotto forma di stringhe, le quali sono inviate e ricevute dal client e dal server ad ogni richiesta e risposta HTTP. Queste intestazioni sono solitamente invisibili all'utente e vengono elaborate e loggate solo dalle rispettive applicazioni del server e del client. Definiscono il modo in cui la connessione e le informazioni inviate/ricevute attraverso essa devono essere interpretate e gestite.

### Headers della richiesta

Di default, le richieste effettuate con `requests` hanno il seguente header:

<pre>
GET https://scrapeme.live/shop/ HTTP/1.1

Host: scrapeme.live
User-Agent: python-requests/2.28.2
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive
</pre>

Per ottenere questa informazione dobbiamo innanzitutto accedere all'oggetto che contiene la richiesta originale. Questo è accessibile tramite l'attributo `.request` della risposta. Infine è sufficiente accedere alla proprietà `.headers` della richiesta originale.

In [2]:
from pprint import pprint
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

pprint(dict(response.request.headers), width=100)

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'User-Agent': 'python-requests/2.28.2'}


> APPROFONDIMENTO: Se avete dei dubbi, e volete osservare il reale header inviato, è necessario "sniffare" la richiesta con un programma esterno. Il modo più semplice è attraverso un proxy che consenta l'analisi del traffico. Per esempio [_**mitmproxy**_](https://mitmproxy.org/) (MITM sta per *Man In The Middle*).

In questo modo ci rendiamo conto che il campo `User-Agent` riporta il valore `python-requests/2.28.2`. Quindi sappiate che se non modifichiamo l'header della richiesta, il server saprò che stiamo usando Python `requests`!

Per modificare il campo `User-Agent`, come qualsiasi altro campo, è sufficiente passare un dizionario contenente le chiavi e i valori che desideriamo.

In [1]:
from pprint import pprint
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:109.0) Gecko/20100101 Firefox/113.0'
}

response = requests.get('https://requests.readthedocs.io/en/master/', headers=headers)

pprint(dict(response.request.headers), width=100)

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:109.0) Gecko/20100101 Firefox/113.0'}


Se volete usare gli user agent più comuni e più recenti, li potete trovare qua:

-  [Firefox](https://www.whatismybrowser.com/guides/the-latest-user-agent/firefox)
-  [Chrome](https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome)

Esistono molti siti che offrono elenchi degli user agent noti.

Tuttavia sappiate che un normale browser invia molti altri dati negli header della richiesta, come per esempio informazioni sui cookie e altre utili al funzionamento del sito. Quindi, se il server a cui vi state collegando necessita di particolari header, dovete provvedere voi a impostarli in modo dorretto prima di effettuare la richiesta.

### Headers della risposta

Anche la risposta contiene degli header. Per visualizzarli, basta accedere all'attributo `.headers` dell'oggetto di risposta:

In [17]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

print(type(response.headers))

display(dict(response.headers))

<class 'requests.structures.CaseInsensitiveDict'>


{'Date': 'Sat, 20 May 2023 10:12:46 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Vary': 'Accept-Encoding',
 'x-amz-id-2': 'QWwr6eRPN4epZLvM+2s7IOAQ8kHovn5hxRuv3H4RnHN71lVF/jWbCQUUyOtNuBJgyumGGXOt7ys=',
 'x-amz-request-id': 'ZVJ2ANGAMTKE7NDK',
 'Last-Modified': 'Mon, 15 May 2023 15:05:40 GMT',
 'ETag': 'W/"3bc64e8dfb9ad78da7dfc4dd7f82af3f"',
 'x-amz-server-side-encryption': 'AES256',
 'x-amz-meta-mtime': '1684163110.360453993',
 'X-Served': 'Nginx-Proxito-Sendfile',
 'X-Backend': 'web-i-09882d230536e047f',
 'X-RTD-Project': 'requests',
 'X-RTD-Version': 'latest',
 'X-RTD-Path': '/proxito/html/requests/latest/index.html',
 'X-RTD-Domain': 'requests.readthedocs.io',
 'X-RTD-Version-Method': 'path',
 'X-RTD-Project-Method': 'public_domain',
 'Referrer-Policy': 'no-referrer-when-downgrade',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload',
 'X-Content-Type-Options': 'nosniff',
 'CDN-Cache-Contr

Viene restituito un oggetto simile a un dizionario, in modo da poter accedere al valore dell'intestazione desiderato tramite la sua chiave. Si noti che le intestazioni sono insensibili alle maiuscole e alle minuscole, il che significa che non ci si deve preoccupare di come sono scritte:

In [19]:
import requests

response = requests.get('https://requests.readthedocs.io/en/master/')

print(response.headers['Content-Type'])
print(response.headers['CONTENT-TYPE'])


text/html; charset=utf-8
text/html; charset=utf-8


### Parametri delle *GET query string*

Una "*query string*" è una convenzione sintattica per aggiungere coppie chiave-valore a un URL.

È interpretate come un'informazione separata dall'URL standard per mezzo di un segno di punto interrogativo `?` e contiene coppie chiave-valore. Ogni chiave è separata dal valore da un segno di uguaglianza =, mentre le coppie sono separate da una e commerciale `&`.

Tramite le *query string* possiamo simulare i dati aggiunti a un URL di base dal browser o da altre applicazioni client. Il modo in cui questi parametri vengono interpretati dipende dall'applicazione lato server. Per esempio, `https://www.python.org/search/` è una pagina di ricerca del sito ufficiale di Python. Se si cerca `'tutorial'`, i risultati saranno visualizzati nella pagina con l'URL `https://www.python.org/search/?q=tutorial`.

Quando si usa `requests`, non è necessario aggiungere manualmente le query string agli URL. La libreria consente di fornire questi argomenti come un dizionario di stringhe utilizzando il *keyword argument* `params` quando si effettua una richiesta:

In [24]:
import requests

# Il dizionario con i parametri della query
my_params = {'q': 'tutorial',
             'o': 1}

# Questa richiesta otterrà la pagina con i risultati della ricerca di "tutorial"
# sul sito ufficiale di Python:
search_res = requests.get('https://docs.python.org/3/search.html', params=my_params)

print(search_res.request.url)
print(search_res.text)

https://docs.python.org/3/search.html?q=tutorial&o=1

<!DOCTYPE html>

<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Search &#8212; Python 3.11.3 documentation</title><meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    <link rel="stylesheet" type="text/css" href="_static/pygments.css" />
    <link rel="stylesheet" type="text/css" href="_static/pydoctheme.css?digest=2d3badd06fe70b34b68db01f99471ce1624ffe4a" />
    
    
    <script data-url_root="./" id="documentation_options" src="_static/documentation_options.js"></script>
    <script src="_static/jquery.js"></script>
    <script src="_static/underscore.js"></script>
    <script src="_static/doctools.js"></script>
    
    <script src="_static/sidebar.js"></script>
    
    <script src="_static/searchtools.js"></script>
    <script src="_static/language_data.js"></script>
    <link rel="search" type="application/opensearc

Se è necessario inviare richieste simili più volte, è opportuno definire una funzione speciale per questo.

Ad esempio, google_search(query, num) restituisce un URL alla pagina contenente un numero di risultati di ricerca di Google per una determinata query:

In [27]:
import requests

def my_search(query, num):
    res = requests.get('https://docs.python.org/3/search.html', params={
        'q': query,
        'num': num
    })
    return res

new_res = my_search('urllib', 1)

print(new_res.url)

https://docs.python.org/3/search.html?q=urllib&num=1


Attenzione a non fare troppe richieste in breve tempo!

- Molte richieste possono portare al blocco del vostro IP o dell'eventuale account che utilizzate per accedere al server. Cercate di essere attenti quando inviate più richieste! Vedremo più avanti alcuni suggerimenti.

Purtroppo il Web oggi non è più solo HTML!

- Oggi la maggior parte dei motori di ricerca sia come Google sia quelli "interni" a ciascun sito utilizzano JavaScript per reperire le richieste dal server e comporre la pagina con i risultati. Questo complica un po' le cose perché non ci basta la pagina HTML di risposta, ma dobbiamo eseguire alcuni script JavaScript necessari al corretto funzionamento della pagina.

## Richiesta POST: `requests.post()`

Una richiesta POST consente di inviare alcuni dati aggiuntivi al server. A differenza delle richieste GET, in cui si possono specificare parametri aggiuntivi nella query string, le richieste POST passano i dati aggiuntivi nel corpo del messaggio (di *request*).

Con la libreria `requests`, è possibile effettuare una richiesta POST utilizzando `requests.post`, specificando i dati aggiuntivi con il parametro `data`, ad esempio come dizionario.

Si immagini di voler aggiungere un nuovo post al proprio blog personale. Si potrebbe scrivere qualcosa di simile:

In [2]:
import requests

my_data = {'post_text': 'Alcuni esempi con Python per fare il caffè!'}

res = requests.post('https://httpbingo.org/post', data=my_data)

print(res)
# print(res.text)

<Response [200]>


## Richiesta PUT e DELETE: `requests.put()` e `requests.delete()`

Il metodo PUT sostituisce la risorsa all'URL dato con la risorsa specificata nella richiesta. Se originariamente non esiste una risorsa da sostituire sul server, PUT ne creerà una.

Si potrebbe anche voler rimuovere una risorsa dal server. Per farlo, occorre inviare una richiesta DELETE, che cancella la risorsa identificata dall'URL della richiesta.

Tuttavia questi due metodi non ci servono attualmente, ai fini del *data retrival*, in quanto i dati vogliamo principalmente leggerli, non cancellarli o aggiornali.

## Idempotenza

Una proprietà importante di alcune richieste HTTP è l'idempotenza. Una richiesta è detta idempotente se l'invio della stessa richiesta più di una volta non introduce ulteriori modifiche allo stato del server. 

Analizzando le richieste menzionate in questa sezione, quali hanno questa proprietà?

Ovviamente, GET è idempotente,o meglio, nullipotente, perché si limita a raccogliere i dati dal server senza introdurre alcuna modifica.

Anche PUT è idempotente, perché se si invia lo stesso oggetto più volte, non avrà alcun effetto rispetto all'invio di una richiesta PUT una sola volta.

E, naturalmente, se si elimina una risorsa una volta, lo stato del server non cambierà ulteriormente quando si invia la stessa richiesta più volte, il che significa che DELETE è idempotente.

Tuttavia, si noti che POST non è idempotente, perché una richiesta POST può aggiungere nuovi dati al server (ad esempio, un nuovo record a un database) e farlo più volte è diverso dall'aggiungerlo una sola volta.

## Uso pratico

In realtà il funzionamento questi metodi HTTP dipendono da come vengono implementate le applicazioni lato server. I nomi di questi tipi di richieste sono assolutamente convenzionali e rappresentano solamente dei modi diversi per inviare dei dati al server. Che cosa il server ne farà di quei dati è un altro paio di maniche. Non c'è nessun'impedimento a usare il metodo DELETE per leggere i dati e il POST per eliminarli.

Naturalmente nessuno stravolge il significato di questi metodi, tuttavia spesso il metodo POST è utilizzato al posto del metodo GET in quanto i parametri della query string sono visibili nell'URL e dunque vengono memorizzati come metadati nei log dei provider internet. Per una maggiore privacy, oggi si tende ad inviare le informazioni considerate più sensibili tramite il metodo POST piuttosto che GET. Inoltre, il contenuto del POST può esere completamente crittografato, cosa che non si può fare del tutto con GET.



## Riassumendo

- `requests` è una libreria Python di terze parti per effettuare richieste HTTP.

GET

- La richiesta `requests.get()` viene utilizzata per recuperare i dati dal server tramite il metodo GET.
- Per fornire parametri aggiuntivi al server con la richiesta GET, utilizzare una query string.
- Una stringa di query può essere passata a `requests.get(url, params)` come un dizionario di coppie chiave-valore.

POST

- POST è il metodo di richiesta più comune utilizzato per inviare dati al server (anche e soprattutto richieste di dati).
- Per effettuare una richiesta POST con requests, utilizzare `requests.post(url, data)`.

PUT e DELETE

- Le richieste PUT sono utilizzate per aggiornare la risorsa sul server. Possono essere inviate con `requests.put(url, data)`.
- La richiesta DELETE `requests.delete(url)` rimuove dal server una risorsa all'URL indicato.

Idempotenza

- Una richiesta HTTP è idempotente se l'invio di più richieste identiche ha lo stesso risultato dell'invio di una singola richiesta.
- GET, PUT e DELETE sono idempotenti, mentre POST non lo è.

## Per fare dei test

Il sito [`httpbingo.org`](https://httpbingo.org/) può essere molto utile quando fai delle prove, perché consente di testare diversi tipi di richieste HTTP.

## Per approfondire:

https://docs.python-requests.org/en/latest/

https://realpython.com/python-requests/

# Modulo built-in `urllib.request`

Il modulo della Libreria standard `urllib.request` offre un'interfaccia per le comunicazioni HTTP a più basso livello e dunque un po' più ostica da usare.

Per approfondire, consiglio di iniziare con i seguenti link:

https://docs.python.org/3/library/urllib.request.html

https://realpython.com/urllib-request/